In [1]:
%env THEANO_FLAGS=mode=FAST_RUN,device=gpu1
import sys
sys.path.insert(0,'../')

import network
reload(network)

net = network.Net()
net.change_lr(0.001)

env: THEANO_FLAGS=mode=FAST_RUN,device=gpu1


Using gpu device 1: Tesla K40c (CNMeM is disabled)


In [2]:
import data_loader
reload(data_loader)

from data_loader import DataLoader

classes = [wnid.strip() for wnid in open('../synsets.txt').readlines()]
dataset = DataLoader( '../../ilsvrc2012','/home/nine/workspace/caffe/data/ilsvrc12/train.txt','/home/nine/workspace/caffe/data/ilsvrc12/val.txt')

In [3]:
from PIL import Image
import preprocessor
reload(preprocessor)
from preprocessor import Fetcher
import tree_distance
reload(tree_distance)
from tree_distance import Distance
import cPickle as pickle

mean = pickle.load(open('mean.pkl'))
fetcher = Fetcher('center',mean)
distance = Distance('../wordnet.is_a.txt','../synsets.txt')

In [9]:
#net.load_model('trained.pkl')
pickle.dump(pickle.load(open('../../models/vgg_cnn_s.pkl'))['values'],open('trained.pkl','wb'))
net.load_model('trained.pkl')

In [10]:
from multiprocessing import Pool, Process, Queue, Value
import sys, datetime

def fetch(q,max_buf,flag,fetcher,distance) :
    while(flag.value != 0) :
        if( q.qsize() < max_buf ) :
            fetched1 = dataset.fetch_train(64)
            fetched2 = dataset.fetch_train(64)

            ims1 = fetcher.fetch(fetched1)
            ims2 = fetcher.fetch(fetched2)
            dist = distance.dist(fetched1,fetched2)
            q.put((ims1,ims2,dist))
            
#             import datetime
#             print 'Enqueued', datetime.datetime.now()
#             sys.stdout.flush()

            
q = Queue()
flag = Value('d', 1) 
p = Process(target=fetch,args=(q,10,flag,fetcher,distance))

p.start()

import logging
logging.basicConfig(filename='train.log',level=logging.DEBUG)

net.change_lr(0.001)
for i in range(100000):
    ims1,ims2,dist = q.get()
    loss = net.train(ims1,ims2,dist)
    
#     import datetime
#     print 'Trained', datetime.datetime.now()
#     sys.stdout.flush()
    if i % 10 == 0 :
        log = "{},{},{}".format(datetime.datetime.now(), i, loss)
        print log
        sys.stdout.flush()
        
        logging.info(log)

flag.value = 0
p.join()

Process Process-1:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-10-e5a79f1db338>", line 10, in fetch
    ims1 = fetcher.fetch(fetched1)
  File "../preprocessor.py", line 44, in fetch
    self._preprocess(loc,ret,i)
  File "../preprocessor.py", line 25, in _preprocess
    im = self.crop(im)
  File "../preprocessor.py", line 36, in _center_crop
    h, w, _ = im.shape
ValueError: need more than 2 values to unpack


KeyboardInterrupt: 

In [ ]:
net.save_model('trained.pkl')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fetch_1 = dataset.fetch_val(1)
fetch_2 = dataset.fetch_val(1)

plt.subplot(121)
plt.imshow(Image.open(fetch_1[0][1]))
plt.subplot(122)
plt.imshow(Image.open(fetch_2[0][1]))

dist,_,_ = net.test(fetcher.fetch(fetch_1),fetcher.fetch(fetch_2))
print classes[fetch_1[0][0]], classes[fetch_2[0][0]], distance.dist(fetch_1,fetch_2), dist